In [1]:
import requests
import re
import pandas as pd
import os
from datetime import date
import datetime
import json

# Settings

In [2]:
read_weather_from_file = False

# Read the data

In [3]:
df_sc = pd.read_csv(os.path.join("data", "sc_export.csv"))

# Parse timestamps

In [4]:
#date,hour = zip(*df_sc['timestamp'].map(lambda x: x.split('T')))
df_sc['timestamp'] = df_sc['timestamp'].map(lambda d:datetime.datetime.strptime(d,'%Y-%m-%dT%H:%M:%S'))

In [24]:
df_sc['day'] = df_sc['timestamp'].map(date.weekday)
df_sc['hour'] = pd.DatetimeIndex(df_sc['timestamp']).hour
df_sc['date'] = pd.DatetimeIndex(df_sc['timestamp']).date
df_sc['time'] = pd.DatetimeIndex(df_sc['timestamp']).hour + pd.DatetimeIndex(df_sc['timestamp']).minute/60 + pd.DatetimeIndex(df_sc['timestamp']).second/3600
df_sc['weekend'] = [1 if d == 5 or d==6 else 0 for d in df_sc['day']]
df_sc['yday'] = df_sc['date'].map(lambda x: x.timetuple().tm_yday)

In [25]:
# get the current day of the year
#doy = datetime.today().timetuple().tm_yday

# "day of year" ranges for the northern hemisphere
spring = range(80, 172)
summer = range(172, 264)
fall = range(264, 355)
# winter = everything else

def get_season(doy):
    if doy in spring:
        season = 'spring'
    elif doy in summer:
        season = 'summer'
    elif doy in fall:
        season = 'fall'
    else:
        season = 'winter'
    return season

df_sc['season'] = [get_season(x) for x in df_sc['yday']]

In [7]:
#df_sc['name'] = [str(lat)+","+str(lon) for lat, lon in zip(df_sc['lat'], df_sc['lon'])]

# Select relevant columns

In [27]:
selected = ["sensor_id", 
            "sensor_type", 
            "location", 
            "lat", "lon", 
            "timestamp", 
            "date", 
            "time", "hour", 
            "day", "weekend", 
            "season",
            "P1", "P2"]

df = df_sc[selected]

# Get weather data

In [9]:
weather_groups = {'Clear': 'dry',
                  'Partially cloudy': 'dry',
                  'Rain, Partially cloudy': 'wet',
                  'Rain': 'wet',
                  'Rain, Overcast': 'wet',
                  'Overcast': 'wet',
                  'Snow, Rain, Partially cloudy': 'wet'}

In [10]:
if not read_weather_from_file:
    with open('api_keys.json') as f:
        API_KEY = json.loads(f.read())['weather']

    lat_lon_s_id = set(zip(df_sc.lat, df_sc.lon, df_sc.sensor_id))
    days = df_sc.date.unique()

    df_weather = -1

    for lat,lon,sensor_id in lat_lon_s_id:
        min_day = min(days)
        max_day = max(days)
        print(min_day, max_day, lat, lon)
        try:
            url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{lat}%2C{lon}/{min_day}/{max_day}?unitGroup=metric&include=days&key={API_KEY}&contentType=csv"

            df2 = pd.read_csv(url)
            df2['sensor_id'] = sensor_id
            if type(df_weather) == int:
                df_weather = df2
            else:
                df_weather = pd.concat([df_weather, df2],ignore_index=True)
        except:
            print("Something is wrong...")

    df_weather.to_csv(os.path.join("data", "weather.csv"), index=False)
else:
    df_weather = pd.read_csv(os.path.join("data", "weather.csv"))


2021-11-29 2022-09-02 46.042 14.592
Something is wrong...
2021-11-29 2022-09-02 46.096 14.47
Something is wrong...


AttributeError: 'int' object has no attribute 'to_csv'

In [11]:
df_weather['weather'] = [weather_groups[x] for x in df_weather["conditions"]]

TypeError: 'int' object is not subscriptable

In [12]:
df_weather.rename(columns={"datetime": "date"}, inplace=True)

AttributeError: 'int' object has no attribute 'rename'

In [13]:
df_weather['date'] = pd.DatetimeIndex(df_weather['date']).date

TypeError: 'int' object is not subscriptable

In [14]:
selected = ["sensor_id","date", "temp", "tempmax", "tempmin", "conditions", "weather"]

In [15]:
df_weather = df_weather[selected]

TypeError: 'int' object is not subscriptable

In [28]:
if type(df_weather) == int:
    df_weather = pd.DataFrame(columns=selected)

# Add weather to sensor data

In [29]:
df = pd.merge(df, df_weather, how='left', on=['sensor_id', 'date'])

In [30]:
df.to_csv(os.path.join("data", "sc_weather.csv"), index=False)

In [31]:
df

,sensor_id,sensor_type,location,lat,lon,timestamp,date,time,hour,day,weekend,season,P1,P2,temp,tempmax,tempmin,conditions,weather
0,67867,SDS011,54672,46.096,14.47,2021-11-29 13:26:10,2021-11-29,13.436111,13,0,0,fall,8.13,6.63,NaN,NaN,NaN,NaN,NaN
1,67867,SDS011,54672,46.096,14.47,2021-11-29 13:28:35,2021-11-29,13.476389,13,0,0,fall,10.68,6.80,NaN,NaN,NaN,NaN,NaN
2,67867,SDS011,54672,46.096,14.47,2021-11-29 13:31:00,2021-11-29,13.516667,13,0,0,fall,8.80,6.70,NaN,NaN,NaN,NaN,NaN
3,67867,SDS011,54672,46.096,14.47,2021-11-29 13:33:26,2021-11-29,13.557222,13,0,0,fall,9.18,6.57,NaN,NaN,NaN,NaN,NaN
4,67867,SDS011,54672,46.096,14.47,2021-11-29 13:35:51,2021-11-29,13.597500,13,0,0,fall,10.43,7.40,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151471,67867,SDS011,54672,46.096,14.47,2022-09-02 23:30:39,2022-09-02,23.510833,23,4,0,summer,5.68,3.28,NaN,NaN,NaN,NaN,NaN
151472,67867,SDS011,54672,46.096,14.47,2022-09-02 23:41:20,2022-09-02,23.688889,23,4,0,summer,11.05,3.75,NaN,NaN,NaN,NaN,NaN
151473,67867,SDS011,54672,46.096,14.47,2022-09-02 23:43:47,2022-09-02,23.729722,23,4,0,summer,7.30,3.47,NaN,NaN,NaN,NaN,NaN
151474,67867,SDS011,54672,46.096,14.47,2022-09-02 23:54:50,2022-09-02,23.913889,23,4,0,summer,19.00,8.38,NaN,NaN,NaN,NaN,NaN
